# Example 33: IoT Windows

Tumbling Windows aggregation.

## 1. Environment Setup

In [ ]:
# Install Java 11
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Install Apache Flink
!pip install -q apache-flink

In [ ]:
from pyflink.common import Types
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.window import TumblingProcessingTimeWindows
from pyflink.common.time import Time
import time

env = StreamExecutionEnvironment.get_execution_environment()
env.set_parallelism(1)

# Simulating continuous stream
# For simplicity here, we stick to collection but Flink windows usually need time
# In Colab 'processing time' is easiest to demo without event-time complexity

# (SensorID, Temp)
data = [
    ("s1", 20), ("s1", 22), ("s2", 15), ("s1", 25), 
    ("s2", 18), ("s1", 23), ("s2", 16), ("s1", 24)
]

ds = env.from_collection(data)

ds.key_by(lambda x: x[0]) \
  .window(TumblingProcessingTimeWindows.of(Time.milliseconds(5))) \n  .reduce(lambda a, b: (a[0], (a[1] + b[1]) / 2)) \n  .print()

# Note: With from_collection, processing time windows might trigger fast.
# In real streaming, we would use a SourceFunction.
env.execute("iot-windows")